In [1]:
model_name = "skt/kogpt2-base-v2"
save_dir = model_name.replace("/", "__")

### PPO 데이터셋 확인

In [5]:
import json
data_path_3_PPO = '/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_3_PPO.jsonl'
with open(data_path_3_PPO, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

print(len(list_data_dict)) # 12000
list_data_dict[:3]

12000


[{'prompt': '번디는 자신이 탐정잡지, 범죄소설 그리고 성범죄 관련 실제 범죄 다큐멘터리들을 탐독했다고 누구에게 말했나?'},
 {'prompt': '개포주공아파트는 몇 단지로 이루어져 있나?'},
 {'prompt': '김영삼의 후보 시절 지역표심을 겨냥한 발언을 문제삼은 후보는?'}]

### 필요한 라이브러리 추가

In [6]:
from copy import deepcopy

import torch
from torch.optim import Adam
from chatgpt.models.base import RewardModel
from chatgpt.models.gpt import GPTActor, GPTCritic
from chatgpt.trainer import PPOTrainer
from chatgpt.trainer.strategies import NaiveStrategy
from transformers import AutoTokenizer

### 모델학습에 사용할 옵티마이저와 모델을 준비

In [8]:
with NaiveStrategy().model_init_context():
    actor = GPTActor(pretrained=f"model/{save_dir}/output_1_SFT", lora_rank=0).to(torch.cuda.current_device())
    critic = GPTCritic(pretrained=f"model/{save_dir}/output_2_RM", lora_rank=0).to(torch.cuda.current_device())

    tokenizer = AutoTokenizer.from_pretrained(
        model_name, bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
        padding_side="right", 
        model_max_length=512
    )

    initial_model = deepcopy(actor)
    reward_model = RewardModel(deepcopy(critic.model), deepcopy(critic.value_head)).to(torch.cuda.current_device())

In [10]:
actor_optim = Adam(actor.parameters(), lr=5e-6)
critic_optim = Adam(critic.parameters(), lr=5e-6)

(actor, actor_optim), (critic, critic_optim), reward_model, initial_model = NaiveStrategy().prepare(
    (actor, actor_optim), (critic, critic_optim), reward_model, initial_model)

### PPO 학습에 쓸 데이터를 불러와 토크나이징

In [11]:
with open('/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_3_PPO.jsonl', "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)
    list_prompt = [tmp['prompt'] for tmp in list_data_dict]

def tokenize_fn(texts):
    batch = tokenizer(texts, return_tensors='pt', max_length=96, padding=True, truncation=True)
    return {k: v.cuda() for k, v in batch.items()}

print(tokenize_fn('It takes something more than intelligence to act intelligently.'))

len(list_prompt)

{'input_ids': tensor([[47311, 10448, 19008,  9792, 11780, 11308, 30190, 10929, 11849, 21663,
         44389,  9574, 13799,   458, 14308, 12778, 22469, 20938, 44696,   458,
         13799,   458, 14308, 12778, 11756, 18944,   389]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1]], device='cuda:0')}


12000

### PPO Trainer 선언

In [12]:
trainer = PPOTrainer(NaiveStrategy(),
                     actor,
                     critic,
                     reward_model,
                     initial_model,
                     actor_optim,
                     critic_optim,
                     max_epochs=1,  
                     train_batch_size=8, 
                     tokenizer=tokenize_fn,
                     max_length=128,
                     do_sample=True,
                     temperature=1.0,
                     top_k=50,
                     pad_token_id=tokenizer.pad_token_id,
                     eos_token_id=tokenizer.eos_token_id)

### PPO 학습 진행

In [13]:
trainer.fit(list_prompt, 
            num_episodes=10,  
            max_timesteps=3,
            update_timesteps=3)

actor.model.save_pretrained(f'./model/{save_dir}/output_3_PPO')

Episode [10/10]: 100%|██████████| 3/3 [00:19<00:00,  6.47s/it]


### RLHF가 적용된 custom chatgpt의 생성능력을 확인

In [14]:
def generation(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device())
    outputs = actor.generate(input_ids,
                             max_length=250,
                             do_sample=True,
                             top_k=50,
                             top_p=0.95,
                             num_return_sequences=1)
    output = tokenizer.batch_decode(outputs[0], skip_special_tokens=True)[0]
    print()
    print(output)
    return output

PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}

list_prompt = [
    '불고기용 고기 한우에요?', 
    '리처드 닉슨이 43대 부통령직을 수행한 년도는?', 
    '시카고 오헤어 국제공항은 어디에 있어',
    '오늘 미세먼지 어때?']

list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt': tmp}) for tmp in list_prompt]

for input_text in list_prompt:
    output = generation(input_text)


### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):'저는 인공지능 어시스턴트이기 때문에 한우 쇠고기를 판매하는 식당이나 가게, 시장에서 사용되는 쇠고기 종류와 종류를 정확히 알 수 없습니다. 하지만 현재 국내에서 사용되는 한우는 육수의 함량이 높아 비싸고 건강에 좋지 않기 때문에 많이 먹는 것으로 알려져 있습니다. 따라서 해당 음식점에서 사용하는 쇠고기를 구입하려면 한우와 육수의 배합비를 비교하면서 충분히 비교 분석하고, 전문가의 조언을 받는 것이 좋습니다.憂泳)입니다.私上品: "Isset too the fast about lower", 'token': 108}憂上品: "It's tell more cores."上官間民的 four: "Is can any and hot curring about land."私上 essry to food): "I'm sorry assering."寶鏡: "I see set made in appropreneuracter.", 'token': 77}報告上 

### Instruction(명령어):
리처드 닉슨이 43대 부통령직을 수행한 년도는?

### Response(응답):'이 질문에 대한 답은 다음과 같습니다.\n\n- 1990- 1990-2- 1996년 12월 9일, 닉슨은 46대 부통령직을 수행한 년도의 기간을 알 수 없습니다. 그러나 닉슨은 2004년 대통령 선거에서 공화당 후보 후보인 빌 클린턴에게 패배했지만, 이번 대선에서 후보들이 출마하면서 부통령직을 수행했다. \n- 닉슨 부통령은 1996년 9월 14일, 대통령 선거의 일환으로 부통령 후보인 존 매케인 상원의원에게 사퇴하면서 대통령직을 수행하였다. \n\n1. 부통령직 수행기간: 1990-1993년 12월 9일, 대통령 선거 때 부통령직 수행기간은 끝났지만, 부통령직에서 물러난 전력이 있는 인물이 아니었기 때문에 부통령직을 수행하지 않았습니다.\n\n2. 대통령 후보인 조지 부시 미 대통령의 재임 중 부통령직은 대통령 후보

#### 메모리 관리를 위해 캐시를 비우기

In [15]:
torch.cuda.empty_cache()